In [1]:
import pandas as pd
import re
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
tqdm.pandas()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
df = pd.read_csv('C:/Stocker_Project/Stocker/data/daum_news_samsung_20230301_20240229.csv')
df_stock = pd.read_csv('C:/Stocker_Project/Stocker/data/samsung_stock_20230301_20240228.csv')
df['date'] = pd.to_datetime(df['날짜']).dt.date

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r'[^\w\s.,!?%+-]', '', str(text))
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_title'] = df['뉴스제목'].apply(clean_text) # 뉴스제목 정제

In [3]:
model_name = "snunlp/KR-FinBERT-SC" # Finbert 모델 지정
tokenizer = AutoTokenizer.from_pretrained(model_name) # 토크나이징
model = AutoModelForSequenceClassification.from_pretrained(model_name)
print(model.config) # 모델 설정 확인
model.to(device) # 모델을 GPU로 이동
label_map = {0: 'negative', 1: 'neutral', 2: 'positive'}

BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.52.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 20000
}



In [4]:
# 감성 분석 함수
def predict_sentiment(text):
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        truncation=True, 
        max_length=128, 
        padding=True
    )
    outputs = model(**inputs)

    probs = torch.nn.functional.softmax(outputs.logits, dim=-1) # 확률값으로 변환
    conf, pred_label = torch.max(probs, dim=1)
    return label_map[pred_label.item()], conf.item()

def get_sentiment_score(text):
    if not text.strip():
        return 0
    label, conf = predict_sentiment(text)

    # 긍부정 확률값 반환
    if label == 'positive':
        return 1 * conf
    elif label == 'negative':
        return -1 * conf
    else:
        return 0


In [5]:
df['sentiment_score'] = df['clean_title'].progress_apply(get_sentiment_score) # 날짜 기준으로 정렬

daily_sentiment = df.groupby('날짜').agg(
    avg_sentiment=('sentiment_score', 'mean'),
).reset_index()

100%|██████████| 18270/18270 [25:25<00:00, 11.97it/s] 


In [6]:
df_stock.dropna(inplace=True) # 결측치 있는 행 삭제

df_stock = df_stock[df_stock['거래량'] > 0] # 거래량 0인 행 삭제

df_stock = df_stock.sort_values('날짜')

In [7]:
df_stock = df_stock.loc[:, ~df_stock.columns.duplicated()]

# 날짜 형식 변호나 & 시간 정규화
daily_sentiment['날짜'] = pd.to_datetime(daily_sentiment['날짜']).dt.normalize()
df_stock['날짜'] = pd.to_datetime(df_stock['날짜']).dt.normalize()

# 날짜 기준으로 두 데이터 내부 조인
df_merged = pd.merge(
    df_stock,
    daily_sentiment,
    on='날짜',
    how='inner',
    suffixes=('_stock', '_sentiment')
)

# 날짜 컬럼 정리 (날짜 겹침)
if '날짜_sentiment' in df_merged.columns:
    df_merged.drop(columns=['날짜_sentiment'], inplace=True)

if '날짜_stock' in df_merged.columns:
    df_merged.rename(columns={'날짜_stock': '날짜'}, inplace=True)

if 'avg_sentiment' in df_merged.columns: # 컬럼명 변경
    df_merged.rename(columns={'avg_sentiment': '감성점수'}, inplace=True)

df_merged['감성점수'] = df_merged['감성점수'].fillna(0) # 결측값 0으로 채우기

df_merged = df_merged.sort_values('날짜')
df_merged['변동률(%)'] = df_merged['변동률(%)'].fillna(0)


In [8]:
df_merged['감성점수'] = df_merged['감성점수'].round(2)
print(df_merged.head())

df_merged.to_csv('data_preprocessing.csv', index=False)


          날짜     종가       거래량  변동률(%)  상승 여부  감성점수
0 2023-03-02  60800  13095682    0.00      0  0.55
1 2023-03-03  60500  10711405   -0.49      0  0.27
2 2023-03-06  61500  13630602    1.65      1  0.50
3 2023-03-07  60700  11473280   -1.30      0  0.16
4 2023-03-08  60300  14161857   -0.66      0 -0.01
